In [ ]:
# 导入必要的函数库
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
import serial
from PIL import Image
import pylab as pl
from scipy import ndimage
import scipy.ndimage
import threading
import time
from matplotlib.patches import Polygon

In [ ]:
# 打开摄像头，占用内存大，不轻易运行
capture=cv2.VideoCapture(0)

In [ ]:
# 获取并处理图像
def get_image():
    while True:
        # 开全局变量处理，分理处红绿蓝
        global image, image_red, image_green, image_blue
        # 从摄像头获取图像
        _,image=capture.read()
        image_red=image[:,:,2].astype(np.float32)
        image_green=image[:,:,1].astype(np.float32)
        image_blue=image[:,:,0].astype(np.float32)
        #cv2.imshow('image',image)
        #cv2.waitKey(1)

# 获取图像单独开进程
get_image_threading = threading.Thread(target=get_image)
get_image_threading.start()

In [ ]:
# 拍照，保存一张照片
def take_photo(photo_name):
    global capture
    photo_name=photo_name+'.jpg'
    _,image=capture.read()
    cv2.imwrite(photo_name,image)
    print('take photo success')

In [ ]:
# 保存一张图片
def take_photo(photo_name, photo):
    photo=photo.astype(np.uint8)
    photo_name=photo_name+'.jpg'
    cv2.imwrite(photo_name,image)
    print('save photo success')

In [ ]:
# 展示图片
def show_image(image):
    image_show=image.astype(np.uint8)
    plt.imshow(image_show, cmap='gray')
    plt.axis('off')  # 不显示坐标轴
    plt.show()

In [ ]:
# 模式匹配获取目标位置，这里都统一采用灰度图
# 这个是目标图像
aim_image = cv2.imread('aim.png')
# 这个是目标图像的灰度图
aim_gray = cv2.cvtColor(aim_image, cv2.COLOR_BGR2GRAY)
threshold = 0.8
def get_aim_point(aim_gray,image_gray):
    global aim_center
    try:
        result = cv2.matchTemplate(image_gray, aim_gray, cv2.TM_CCOEFF_NORMED)
        locations = np.where(result >= threshold)
        # 转换为(x, y)坐标
        locations = list(zip(*locations[::-1]))
        # 转换为目标中心点坐标
        aim_center=np.array([int(locations[0][1] + aim_image.shape[0] / 2),int(locations[0][0] + aim_image.shape[1] / 2)])
    except:
        aim_center=aim_center
    return(aim_center)

In [ ]:
# 突出图像中的红色目标
def get_red():
    global image, image_red, image_green, image_blue
    image_red_only=image_red*2-image_blue-image_green
    image_red_only[image_red_only<0]=0
    return image_red_only

In [ ]:
# 突出图像中的绿色目标
def get_green():
    global image, image_red, image_green, image_blue
    image_green_only=image_green*2-image_blue-image_red
    image_green_only[image_green_only<0]=0
    return image_green_only

In [ ]:
# 突出图像中的蓝色目标
def get_blue():
    global image, image_red, image_green, image_blue
    image_blue_only=image_blue*2-image_blue-image_red
    image_blue_only[image_blue_only<0]=0
    return image_blue_only

In [ ]:
# 在某一图像的指定位置添加图案
kernel = np.array([
[0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0],
[0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0],
[0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0],
[0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0],
[0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0],
[0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0],
[0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0],
[0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0],
[0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0],
[0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0],
[0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0],
[0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0],
[0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0],
])*255
def draw_image(image_gray_to_draw, kernel, kernel_point):
    try:
        image_gray_to_draw[int(kernel_point[0]-11):int(kernel_point[0]+11),int(kernel_point[1]-11):int(kernel_point[1]+11)]=kernel
    except:
        pass
    return image_gray_to_draw

In [ ]:
# 图像的映射变换
def photo_transform(ad,origin,border_point):
    # 原先的四个角的边缘点
    pts = np.float32([[0,0],[0,origin.shape[1]],[origin.shape[0],origin.shape[1]],[origin.shape[0],0]])
    # 由于变换的特殊性，这里后面的点是列行，反着来的
    border_point_1=np.zeros(np.shape(border_point))
    border_point_1[:,0]=border_point[:,1]
    border_point_1[:,1]=border_point[:,0]
    pts1 = np.float32(border_point_1)
    # 生成最为关键的变换矩阵
    M = cv2.getPerspectiveTransform(pts,pts1)
    # 变换正方形
    square_trans = cv2.warpPerspective(origin,M,(ad.shape[1],ad.shape[0]))
    square_trans = cv2.cvtColor(square_trans, cv2.COLOR_RGB2GRAY)
    _, square_trans = cv2.threshold(square_trans, 127, 255, cv2.THRESH_BINARY)
    return square_trans

# 上面的那个表述不清，具体参考下面这个
# 我们需要将我们读取到的图像映射成一个平整的图像
# 对应原先图像的，左上，左下，右下，右上，四个点
pts0=np.float32([[63,165],[413,116],[413,562],[53,502]])
# 建立映射需要的一一对应
pts1=np.float32([[pts0[0][1],pts0[0][0]],[pts0[3][1],pts0[3][0]],[pts0[2][1],pts0[2][0]],[pts0[1][1],pts0[1][0]]])
# 对应映射后图像，左上，左下，右下，右上，四个点
pts2=np.float32([[0,0],[968,0],[968,968],[0,968]])
# 生成最为关键的变换矩阵
M = cv2.getPerspectiveTransform(pts1,pts2)
# 变换正方形
square_trans = cv2.warpPerspective(ex1_grey,M,(grid.shape[1],grid.shape[0]))
show_image(square_trans)

In [ ]:
# 这一步用于识别黑色条纹的位置，计算误差后传给stm32
def get_road_error(road):
    # 100:540
    global error,k
    d1=np.argwhere(road[70:100,100:540] < 250)
    dis1=np.mean(d1,axis=0)-220
    d2=np.argwhere(road[250:280,100:540] < 250)
    dis2=np.mean(d2,axis=0)-220
    d3=np.argwhere(road[400:420,100:540] < 250)
    dis3=np.mean(d3,axis=0)-220
    dis=0.2*dis1+0.5*dis2+0.3*dis3
    k=dis[1]
    k=min(max(-500,k),500)
    if k>=0:
        error='E'+'+'+str(int(int(k)%1000/100))+str(int(int(k)%100/10))+str(int(k)%10)
    if k<0:
        error='E'+'-'+str(int(int((-k))%1000/100))+str(int(int((-k))%100/10))+str(int((-k))%10)
    return error,k

In [ ]:
# 找寻线路，找出要巡迹的那一条线
def get_road():
    # 先得到排除其他干扰后的路线
    road_only=get_red()
    # 这里可能是别的颜色的线
    road_only

In [ ]:
# 测试摄像头刷新率
def test_fps():
    time1=time.time()
    for i in range(100):
        _,image=capture.read()
        # 在此加入别视觉处理的函数
    time2=time.time()
    print('fps:',100/(time2-time1))

In [ ]:
# 做映射变换（输入的图像为3色图，结果为黑白图，这里也可以自我调整）
def image_trans(image_to_trans):
    image_after_trans = cv2.warpPerspective(image_to_trans,M,(ad.shape[1],ad.shape[0]))
    image_after_trans=cv2.cvtColor(image_after_trans, cv2.COLOR_BGR2GRAY)
    _,image_after_trans=cv2.threshold(image_after_trans, 127, 255, cv2.THRESH_BINARY)
    return image_after_trans

In [ ]:
from Vision_Net import FastestDet
loo_global=np.zeros((640,480,3),dtype=np.uint8)
deep = FastestDet(drawOutput=True)
# 获取照片数字，
def get_photo_number(frame):
    out = deep.detect(frame)
    num_position=[]
    num=[]
    for i in range(len(out)):
        num_position.append(out[i][0])
        num.append(str(out[i][1]))
    cv2.imwrite('change.jpg',frame)
    return  num_position,num

In [ ]:
# 获取当前地图的数字（输入结果为3色图像）
grid_str_int={'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'A':10,'B':11,'C':12,'D':13,'E':14,'F':15,'N':16}
def get_grid_num(image_tra):
    grid_now_str=['N']*16
    for i in range(16):
        x=int(i/4)*242
        y=int(i%4)*242
        pi=image_tra[x+20:x+222,y+20:y+222].astype(np.uint8)
        pi3=np.zeros((202,202,3),dtype=np.uint8)
        pi3[:,:,0]=pi
        pi3[:,:,1]=pi
        pi3[:,:,2]=pi
        _,num=get_photo_number(pi3)
        try:
            grid_now_str[i]=num[0]
        except:
            pass
    return grid_now_str

In [ ]:
# 分割图像操作（一般用于切割图像给神经网络训练）
s=1
for j in range(1,31):
    image=cv2.imread(f'{j}.jpg')
    image_tra=image_trans(image)
    for i in range(16):
        x=int(i/4)*242
        y=int(i%4)*242
        pi=image_tra[x+20:x+222,y+20:y+222]
        cv2.imwrite(f'photo\{s}.jpg',pi)
        s+=1

In [ ]:
# 将图像中的点做映射变换（image_gray这里为隔离红色后的单色图）
# 颜色识别找车
M=np.load('M.npy')
def find_car_red(image_gray):
    global car_center
    try:
        d1=np.argwhere(image_gray>100)
        red1=np.mean(d1,axis=0)
        red2=np.array([red1[1],red1[0],1])
        # 这里相当于是把一个单色的红点给映射变换过去了
        red3=np.dot(M,red2)
        car_center=[red3[1]/red3[2],red3[0]/red3[2]]
    except:
        car_center=car_center
    return car_center

In [ ]:
# 路径规划函数，用于寻找方格路径
grid_cross=[] # 这一位用于记录每个方格的相邻方格
def find_road(aim_point22, grid_coverd_int):
    road22=[[0]]
    road_final=[]
    while aim_point22 not in road_final:
        try:
            if len(road22)>1000:
                print('no way')
                return 0
            if aim_point22 not in grid_coverd_int:
                print('aim_point22 not in grid_coverd_int')
                return 0
            # 用于记录所有可能路径的最后一位
            road_final=[]
            # i表示保存的一条可能路径
            for i in road22:
                # print('i='+str(i))
                # 找出这条路径的最后一位，找出最后一位的所有可能下一位
                for j in grid_cross[i[-1]]:
                    # print('j='+str(j))
                    k=i.copy()
                    if (j not in k) and (j not in road_final) and (j in grid_coverd_int):
                        k.append(j)
                        road22.append(k)
                        road_final.append(j)
        except:
            print('break')
            return 0
    # print(road22)
    road_possible=[]
    road_length=[]
    for r in road22:
        if r[-1]==aim_point22:
            road_possible.append(r)
            road_length.append(len(r))
    # 找出possible_road中的最短路径
    # 找出最短路径
    road_final=road_possible[road_length.index(min(road_length))]
    return road_final